In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList
import torch
import json, re, ast
from pprint import pprint

In [2]:
model_name = "nis12ram/HindiNER-4B-v0.0" ## or "nis12ram/HindiNER-4B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
ner_user_msg = '''You are a Hindi language expert who specializes in extracting entities from text. Given a piece of text, extract all crucial entities along with their respective context-aware entity types. Ensure that entity type is in Hindi. The output should be in JSON format.

## Output format:
```json
{{
  "entities": [
    {{
      "type": "_",
      "value": ["_", "_"]
    }},
    {{
      "type": "_",
      "value": ["_"]
    }}
  ]
}}
```

## Text:
""" {text} """'''


prompt_format = '''<extra_id_0>System

<extra_id_1>User
{user_msg}
<extra_id_1>Assistant
'''

In [4]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_token = input_ids[0][-1]
        for stop in self.stops:
            if tokenizer.decode(stop) == tokenizer.decode(last_token):
                return True
        return False
stop_words = ["<extra_id_1>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [5]:
def extract_json(text):
    # Regex to extract content between ```json and ```
    match = re.search(r'```json\s*([\s\S]*?)\s*```', text)

    if match:
        json_str: str = match.group(1).strip()
        try:
            ## for proper json structure
            return json.loads(json_str)
        except:
            try:
              ## for malformed json
              python_literal: Any = ast.literal_eval(json_str) ## any python literal(str or dict or list or tuple ,..)
              return json.loads(
                  json.dumps(python_literal, ensure_ascii=False) ## json encoded string
              ) ## json data loaded as python literal
            except:
              print(f"NOT ABLE TO EXTRACT JSON DATA FROM TEXT: {text}")
              return None

    print(f"NOT ABLE TO EXTRACT JSON DATA FROM TEXT: {text}")
    return None

In [6]:
from typing import Dict, Any
def inference(text: str, max_new_tokens: int=1000, sampling_params: Dict[str, Any]={}):
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=max_new_tokens,
      **sampling_params,
      stopping_criteria=stopping_criteria
  )
  output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

  content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
  content = extract_json(content)
  return content

In [7]:
text = """एअर इंडिया ने X पर बताया है कि गुजरात के अहमदाबाद एयरपोर्ट के पास गुरुवार को हुए विमान हादसे में फ्लाइट में सवार 242 लोगों में से 241 लोगों की मौत हो गई और सिर्फ एक शख्स जीवित बचा है। एअर इंडिया ने बताया, 'यात्रियों में 169 भारतीय नागरिक, 53 ब्रिटिश और 7 पुर्तगाली और 1 कनाडाई नागरिक था।'"""

In [8]:
input_text = prompt_format.format(user_msg=ner_user_msg.format(text=text))
json_output = inference(input_text, max_new_tokens=2000, sampling_params={"do_sample": False})
pprint(json_output)

{'entities': [{'type': 'विमान कंपनी', 'value': ['एअर इंडिया']},
              {'type': 'स्थान', 'value': ['गुजरात', 'अहमदाबाद']},
              {'type': 'देश',
               'value': ['भारत', 'ब्रिटेन', 'पुर्तगाल', 'कनाडा']},
              {'type': 'नागरिकता',
               'value': ['भारतीय नागरिक',
                         'ब्रिटिश नागरिक',
                         'पुर्तगाली नागरिक',
                         'कनाडाई नागरिक']},
              {'type': 'विमान हादसा', 'value': ['गुरुवार को हुए विमान हादसे']},
              {'type': 'व्यक्ति संख्या',
               'value': ['242 लोगों', '241 लोगों', '1 शख्स']}]}
